# Lab | Chains in LangChain

## Outline

* LLMChain
* Sequential Chains
  * SimpleSequentialChain
  * SequentialChain
* Router Chain

In [1]:
#!pip install langchain

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

OPENAI_API_KEY  = os.getenv('OPENAI_API_KEY')
HUGGINGFACEHUB_API_TOKEN = os.getenv('HUGGINGFACEHUB_API_TOKEN')

In [4]:
#!pip install pandas
#!pip install langchain-openai

In [5]:
import pandas as pd
df = pd.read_csv('./data/Data.csv')

In [6]:
df.head()


,Product,Review
0,Queen Size Sheet Set,I ordered a king size set. My only criticism w...
1,Waterproof Phone Pouch,"I loved the waterproof sac, although the openi..."
2,Luxury Air Mattress,This mattress had a small hole in the top of i...
3,Pillows Insert,This is the best throw pillow fillers on Amazo...
4,Milk Frother Handheld\r\n,I loved this product. But they only seem to l...


## LLMChain

In [7]:
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain

In [8]:
#Replace None by your own value and justify
llm = ChatOpenAI(temperature=1.5)


In [9]:
#Write a query that would take a variable to describe any product
prompt = ChatPromptTemplate.from_template( "Write a product summary for {product}. "  
    "write about highlights and issues. Be short and concise"
)

In [10]:
# old code
# chain = LLMChain(llm=llm, prompt=prompt)

# Corrected code
chain = prompt | llm

In [11]:
product = "Kindle"
chain.invoke(product)

AIMessage(content="The Kindle is a portable e-reader that provides access to thousands of books, magazines, and newspapers. With a high-resolution display and long battery life, it's perfect for avid readers on the go. However, some users may find the lack of a color display limiting for certain types of content.", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 59, 'prompt_tokens': 24, 'total_tokens': 83, 'completion_tokens_details': {'audio_tokens': None, 'reasoning_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-3fef7bc9-d807-40a1-bec4-58d700c10e02-0', usage_metadata={'input_tokens': 24, 'output_tokens': 59, 'total_tokens': 83})

## SimpleSequentialChain

In [12]:
from langchain.chains import SimpleSequentialChain

In [13]:
llm = ChatOpenAI(temperature=0.9)

# prompt template 1
first_prompt = ChatPromptTemplate.from_template(
    "Write a product summary for {product}. "  
    "write about highlights and issues. Be short and concise"
    #Repeat of the initial query 
)

# Chain 1
chain_one = LLMChain(llm=llm, prompt=first_prompt)

C:\Users\KK\AppData\Local\Temp\ipykernel_18996\458262902.py:11: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain_one = LLMChain(llm=llm, prompt=first_prompt)


In [14]:

# prompt template 2
second_prompt = ChatPromptTemplate.from_template(
    "Based on the review {review}, rate the quality of the product from zero to five"
    #Write the second prompt query that takes an input variable whose input will come from the previous prompt"
)
# chain 2
chain_two = LLMChain(llm=llm, prompt=second_prompt)

In [15]:
# USE OF 'SimpleSequentialChain' to combine 2 prompts
overall_simple_chain = SimpleSequentialChain(chains=[chain_one, chain_two],
                                             verbose=True
                                            )

In [16]:
overall_simple_chain.invoke(product)



> Entering new SimpleSequentialChain chain...
The Kindle is a popular e-reader that allows users to easily download and read books, magazines, and newspapers. With a lightweight design and long battery life, it is perfect for reading on-the-go. The high-resolution display and adjustable font sizes make reading easy on the eyes. However, some users have reported issues with the device freezing or glitches in the software. Overall, the Kindle is a great option for avid readers looking for a convenient way to access a wide variety of reading materials.
I would rate the Kindle e-reader a 4 out of 5. It offers a lot of great features for readers such as a lightweight design, long battery life, high-resolution display, and adjustable font sizes. However, the reported issues with freezing and glitches in the software bring down the overall quality slightly.

> Finished chain.


{'input': 'Kindle',
 'output': 'I would rate the Kindle e-reader a 4 out of 5. It offers a lot of great features for readers such as a lightweight design, long battery life, high-resolution display, and adjustable font sizes. However, the reported issues with freezing and glitches in the software bring down the overall quality slightly.'}

**Repeat the above twice for different products**

## SequentialChain

In [17]:
from langchain.chains import SequentialChain

In [18]:
llm = ChatOpenAI(temperature=0.9)


first_prompt = ChatPromptTemplate.from_template(
  "Translate this review {review} into german"
  #This prompt should translate a review
)

chain_one = LLMChain(llm=llm, prompt=first_prompt, 
                     output_key='ger_review' #Give a name to your output
                    )


In [19]:
second_prompt = ChatPromptTemplate.from_template(
    "summarize this review {review} in less than 10 words in german."
    #Write a promplt to summarize a review
)

chain_two = LLMChain(llm=llm, prompt=second_prompt, 
                     output_key='ger_short_review' #give a name to this output
                    )


In [20]:
# prompt template 3: translate to english or other language
third_prompt = ChatPromptTemplate.from_template(
    "Translate {ger_short_review} into English"
)
# chain 3: input= Review and output= language
chain_three = LLMChain(llm=llm, prompt=third_prompt,
                       output_key='en_review'
                      )


In [21]:

# prompt template 4: follow up message that take as inputs the two previous prompts' variables
fourth_prompt = ChatPromptTemplate.from_template(
        "Rate the quality of the translation from {ger_short_review} into {en_review} from zero to ten."
)
chain_four = LLMChain(llm=llm, prompt=fourth_prompt,
                      output_key='rate_trans'
                     )


In [22]:
# overall_chain: input= Review 
# and output= English_Review,summary, followup_message
overall_chain = SequentialChain(
    chains=[chain_one, chain_two, chain_three, chain_four],
    input_variables=["review"],
    output_variables=['ger_review', 'ger_short_review', 'en_review', 'rate_trans'],
    verbose=True
)

In [23]:
review = df.Review[2]
overall_chain.invoke(review)



> Entering new SequentialChain chain...

> Finished chain.


{'review': "This mattress had a small hole in the top of it (took forever to find where it was), and the patches that they provide did not work, maybe because it's the top of the mattress where it's kind of like fabric and a patch won't stick. Maybe I got unlucky with a defective mattress, but where's quality assurance for this company? That flat out should not happen. Emphasis on flat. Cause that's what the mattress was. Seriously horrible experience, ruined my friend's stay with me. Then they make you ship it back instead of just providing a refund, which is also super annoying to pack up an air mattress and take it to the UPS store. This company is the worst, and this mattress is the worst.",
 'ger_review': 'Diese Matratze hatte ein kleines Loch oben drauf (hat ewig gedauert, um herauszufinden, wo es war), und die mitgelieferten Flicken haben nicht funktioniert, vielleicht weil es die Oberseite der Matratze ist, wo es eher wie Stoff ist und ein Flicken nicht haften bleibt. Vielleich

**Repeat the above twice for different products or reviews**

## Router Chain

In [24]:
physics_template = """You are a very smart physics professor. \
You are great at answering questions about physics in a concise\
and easy to understand manner. \
When you don't know the answer to a question you admit\
that you don't know.

Here is a question:
{input}"""


math_template = """You are a very good mathematician. \
You are great at answering math questions. \
You are so good because you are able to break down \
hard problems into their component parts, 
answer the component parts, and then put them together\
to answer the broader question.

Here is a question:
{input}"""

history_template = """You are a very good historian. \
You have an excellent knowledge of and understanding of people,\
events and contexts from a range of historical periods. \
You have the ability to think, reflect, debate, discuss and \
evaluate the past. You have a respect for historical evidence\
and the ability to make use of it to support your explanations \
and judgements.

Here is a question:
{input}"""


computerscience_template = """ You are a successful computer scientist.\
You have a passion for creativity, collaboration,\
forward-thinking, confidence, strong problem-solving capabilities,\
understanding of theories and algorithms, and excellent communication \
skills. You are great at answering coding questions. \
You are so good because you know how to solve a problem by \
describing the solution in imperative steps \
that a machine can easily interpret and you know how to \
choose a solution that has a good balance between \
time complexity and space complexity. 

Here is a question:
{input}"""

default_template = """You are a helpful assistant. 
Provide a clear, helpful answer.

Here is a Question: 
{input}"""

In [51]:
prompt_infos = [
    {
        "name": "physics", 
        "description": "Good for answering questions about physics", 
        "prompt_template": physics_template
    },
    {
        "name": "math", 
        "description": "Good for answering math questions", 
        "prompt_template": math_template
    },
    {
        "name": "history", 
        "description": "Good for answering history questions", 
        "prompt_template": history_template
    },
    {
        "name": "computer_science", 
        "description": "Good for answering computer science questions", 
        "prompt_template": computerscience_template
    },
    {
        "name": "DEFAULT",
        "description": "Good for general questions",
        "prompt_template": default_template
    }
]

In [52]:
from langchain.chains.router import MultiPromptChain
from langchain.chains.router.llm_router import LLMRouterChain,RouterOutputParser
from langchain.prompts import PromptTemplate

from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from langchain_openai import ChatOpenAI

from operator import itemgetter
from typing import Literal
from typing_extensions import TypedDict

In [53]:
llm = ChatOpenAI(temperature=0)

In [54]:
# Create expert chains
expert_chains = {}
for info in prompt_infos:
    prompt = ChatPromptTemplate.from_template(template=info["prompt_template"])
    chain = prompt | llm | StrOutputParser()
    expert_chains[info["name"]] = chain


In [55]:
# Create router prompt
destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)

router_template = """Given the following question, select the most appropriate category.
Available categories:
{destinations}

Select exactly one category from: physics, math, history, computer_science, or DEFAULT.
Respond with only the category name.

Question: {question}"""

router_prompt = ChatPromptTemplate.from_template(router_template)

In [56]:
# Define router output type
class RouterOutput(TypedDict):
    destination: Literal["physics", "math", "history", "computer_science", "DEFAULT"]

# Create router chain
router_chain = router_prompt | llm.with_structured_output(RouterOutput) | itemgetter("destination")

In [57]:
# Create final chain
def route_to_chain(inputs: dict) -> str:
    destination = inputs["destination"]
    query = inputs["query"]
    return expert_chains[destination].invoke({"input": query})

chain = (
    {
        "query": RunnablePassthrough(),
        "destination": lambda x: router_chain.invoke({"destinations": destinations_str, "question": x})
    }
    | RunnableLambda(route_to_chain)
)

if __name__ == "__main__":
    # Test with different types of questions
    questions = [
        "What is black body radiation?", 
        "What is 2+2",      
        "What caused World War I?",       
        "Explain Big O notation",     
        "What is the forecast for today?", 
        "Explain what principle component analysis is using linear algebra",
       # "What are cells?" #still breaks it 
    ]
    
    for question in questions:
        print(f"\nQuestion: {question}")
        result = chain.invoke(question)
        print(f"Answer: {result}\n")
        print("-" * 50)


Question: What is black body radiation?
Answer: Black body radiation refers to the electromagnetic radiation emitted by a perfect black body, which is an idealized physical body that absorbs all incident electromagnetic radiation. The radiation emitted by a black body depends only on its temperature and follows a specific distribution known as Planck's law. This radiation is characterized by a continuous spectrum of wavelengths and intensities, with the peak intensity shifting to shorter wavelengths as the temperature of the black body increases. Black body radiation plays a key role in understanding concepts such as thermal radiation and the quantization of energy in quantum mechanics.

--------------------------------------------------

Question: What is 2+2
Answer: Thank you for the compliment! The answer to the question "What is 2+2?" is 4. This can be solved by simply adding the two numbers together.

--------------------------------------------------

Question: What caused World

In [58]:
chain.invoke("what is 2 + 2")

'The answer to 2 + 2 is 4.'

In [60]:
chain.invoke("Explain what principle component analysis is using linear algebra")

'Principal component analysis (PCA) is a technique used in statistics and data analysis to reduce the dimensionality of a dataset while preserving as much variance as possible. In linear algebra terms, PCA involves finding the eigenvectors and eigenvalues of the covariance matrix of the dataset.\n\nTo perform PCA using linear algebra, we first center the data by subtracting the mean of each feature from the dataset. Then, we calculate the covariance matrix of the centered data. The covariance matrix represents the relationships between the different features in the dataset.\n\nNext, we find the eigenvectors and eigenvalues of the covariance matrix. The eigenvectors represent the directions of maximum variance in the data, while the eigenvalues represent the amount of variance explained by each eigenvector.\n\nFinally, we can project the data onto the eigenvectors with the highest eigenvalues to reduce the dimensionality of the dataset. This allows us to represent the data in a lower-di

**Repeat the above at least once for different inputs and chains executions - Be creative!**